In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns.
drop_columns = ['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS', 'AFFILIATION', 'USE_CASE']
application_df = application_df.drop(drop_columns, 1)
application_df.head()

<ipython-input-2-54bb454ef614>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(drop_columns, 1)


,APPLICATION_TYPE,CLASSIFICATION,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,Association,1,0,5000,1
1,T3,C2000,Co-operative,1,1-9999,108590,1
2,T5,C3000,Association,1,0,5000,0
3,T3,C2000,Trust,1,10000-24999,6692,1
4,T3,C1000,Trust,1,100000-499999,142590,1


In [3]:
# Choose a cutoff value and create a list of application types to be replaced.  Use the variable name `application_types_to_replace`.
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced. Use the variable name `classifications_to_replace`.
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts <1000].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
application_encoded = pd.get_dummies(application_df)

In [6]:
# Split our preprocessed data into our features and target arrays
X = application_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Compile, Train and Evaluate the Model

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nfeatures = X_train.shape[1]
layer1 = 8
layer2 = 2
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer1, input_dim=nfeatures, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 248       
                                                                 
 dense_1 (Dense)             (None, 2)                 18        
                                                                 
 dense_2 (Dense)             (None, 1)                 3         
                                                                 
Total params: 269 (1.05 KB)
Trainable params: 269 (1.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# Train the model
nn_trained = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.6671 - accuracy: 0.5952
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6587 - accuracy: 0.6055
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6558 - accuracy: 0.6105
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6552 - accuracy: 0.6123
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6516 - accuracy: 0.6207
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6361 - accuracy: 0.6444
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6354 - accuracy: 0.6453
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6347 - accuracy: 0.6459
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6345 - accuracy: 0.6455
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6342 - accura

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6354 - accuracy: 0.6469 - 462ms/epoch - 2ms/step
Loss: 0.6353870630264282, Accuracy: 0.6468804478645325


In [12]:
# Export your model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization_01.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
